# Gráficos geoespaciales: mapas

Habitualmente necesitamos pintar datos que tienen una referencia geoespacial, por ejemplo:

* Puntos en mapas
* Rutas
* Estados, provincias, países u otras divisiones administrativas de territorios

Para hacerlo, podemos utilizar la librería `folium`, que permite utilizar [`leaflet`](https://leafletjs.com/) dentro de python.

Puedes consultar la documentación de folium [aquí](https://python-visualization.github.io/folium/).

In [ ]:
import folium
import pandas as pd

Si te falla el `import folium` es que no tienes la librería instalada. Para hacerlo, corre desde tu terminal:

```
# si usas conda (gestor de paquetes de Anaconda)
conda install -c conda-forge folium

# si usas pip
pip install folium
```

Una vez lo instalas, tendrás que reiniciar el kernel de jupyter antes de volver a probar. Puedes hacerlo desde este notebook en el menú Kernel / Restart.

## Un primer mapa

Para pintar un mapa, llamamos a folium con las opciones que queramos. Por ejemplo, vamos a pintar un mapa de la zona de República Argentina, Madrid

In [ ]:
rep_argentina = [40.4438, -3.6857]
folium.Map(location=rep_argentina, zoom_start=15)

Las imágenes de las calles, el mapa base, se denominan `tiles`. Podemos personalizarlo con el parámetro `tiles`. Uno que suele quedar bastante bien para pintar datos encima es `cartodbpositron`:

In [ ]:
folium.Map(location=rep_argentina, zoom_start=15, tiles='cartodbpositron')

## Marcadores

Para añadir una localización, añadimos uno o varios marcadores al mapa. Sobre ellos, podemos personalizar:

* Localización
* Textos en tooltip o popup
* Estilo (forma, color, ...)

Mira más opciones en la [documentación](https://python-visualization.github.io/folium/modules.html#folium.map.Marker).

In [ ]:
rep_argentina = [40.446, -3.68]
m = folium.Map(location=rep_argentina, zoom_start=15)
folium.Marker([40.4438, -3.6857], tooltip='EAE Joaquín Costa').add_to(m)
folium.Marker([40.4485,-3.6796], tooltip='EAE Príncipe de Vergada').add_to(m)
m

Vamos a añadir una serie de localizaciones a nuestro mapa. En `dat/` tenemos disponibles las localizaciones de los apartamentos de AirBnB en Madrid, descargados de [Inside AirBnB](http://insideairbnb.com/).

In [ ]:
listings = pd.read_csv('dat/listings.csv')
listings.head()

In [ ]:
# Mi mapa base sobre Madrid
map_madrid = folium.Map(location=[40.42, -3.7], zoom_start=13)

# Cojo 100 apartamentos (la muestra completa es muy grande)
listings_sample = listings.sample(100, random_state=1234)

# Los añado a mi mapa
# Voy a usar CircleMarker en lugar de Marker, porque me deja personalizar el color más fácilmente
listings_sample.apply(
    lambda row: folium.CircleMarker([row.latitude, row.longitude], tooltip=row.room_type, radius=3).add_to(map_madrid),
    axis=1
)
map_madrid

#### Ejercicio

Personaliza el mapa anterior para que el color del marcador diferencie por `room_type`.

## Heatmaps

Son útiles para representar densidades.

Vamos a utilizarlo para ver cuál es la concentración de apartamentos por zona.

Si consultamos la [documentación](https://python-visualization.github.io/folium/plugins.html#folium.plugins.HeatMap), vemos que necesita un parámetro `data` que debe ser una lista de `[lat, lng]`, pero también acepta un dataframe de pandas con 2 columnas.

In [ ]:
from folium.plugins import HeatMap

In [ ]:
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')
heatmap = HeatMap(data=listings[["latitude", "longitude"]], radius=15).add_to(m)
m

## Mapa de coropletas

Son mapas sobre los que coloreamos las regiones que contiene en base a la propiedad que queremos explicar.

Vamos a pintar un mapa que represente el precio medio de los alojamientos de cada barrio.

In [ ]:
num_per_area = listings.groupby('neighbourhood').id.count().reset_index()
num_per_area = num_per_area.rename(columns={"id": "num"})
num_per_area.head()

In [ ]:
# Incializamos el mapa
m = folium.Map(location=[40.42, -3.7], zoom_start=11, tiles='cartodbpositron')

# Add the color for the chloropleth:
folium.Choropleth(
    geo_data='dat/neighbourhoods.geojson',
    data=num_per_area,
    columns=['neighbourhood', 'num'],
    key_on='feature.properties.neighbourhood',
    fill_color='YlGn'
).add_to(m)

m

#### Ejercicio

Modifica el mapa anterior para definir los límites de cada color y ver mejor el detalle. P.e. para que de 0 a 10 se vea en una intensidad, de 10 a 100 en otra, de 100 a 1000, ... (es decir, una escala logarítmica)